In [ ]:
# Use python to pull some events form the opensearch
# Install Neo4j in ubuntu 
# Parse the opensearch retrived data and store these data in the Graph database
# retrive the data in the Graph (UI)

In [139]:
!pip install opensearch-py
!pip install neo4j


[notice] A new release of pip is available: 24.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/313.2 kB ? eta -:--:--
   ---------------------------------------- 313.2/313.2 kB 9.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from opensearchpy import OpenSearch
import json

host = 'coreserver'
port = 9200
auth = ('admin', 'kBFXbci#bf6jB9TAe8^fts') # For testing only. Don't store credentials in code.

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = False,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

In [75]:
# Extract All process create events (sysmon: process_create) 
# TODO: now we are pulling all data, but we pull and injest data with 1 or two minute windows

query = {
  'size': 1,
  'from': 0,
  "query": {
    "query_string": {
       "query": "EventID:'1' AND ProviderName:'Microsoft-Windows-Sysmon'"
     }
  }
}

response = client.search(
    body = query,
    index = 'fluentd.events.*'
)

print(f"Number of documents received: {response['hits']['total']}")

max_records = response['hits']['total']["value"]

step = 1000

all_responses = []

for i in range(0, max_records, step ):
    query = {
      'size': step,
      'from': i,
      "query": {
        "query_string": {
          "query": "EventID:'1' AND ProviderName:'Microsoft-Windows-Sysmon'"
        }
      }
    }

    response = client.search(
        body = query,
        index = 'fluentd.events.*'
    )
    all_responses.append(response)
    print(f"Number of documents received in this query: {len(response['hits']['hits'])}")


Number of documents received: {'value': 2547, 'relation': 'eq'}
Number of documents received in this query: 1000
Number of documents received in this query: 1000
Number of documents received in this query: 547


In [ ]:
def strip_paranthesis(id_):
    return id_.strip("}").strip("{")

data_ = []
for resp in all_responses:
    for item in response["hits"]["hits"]:
        source_ = item['_source']
        Parse_data = {}
        for key_ in source_.keys():
            if key_ in ("ProcessGuid","LogonGuid","ParentProcessGuid"):
                Parse_data[key_] = strip_paranthesis(source_[key_]) 
                continue
            if key_ == "Hashes":
                for hash_ in source_[key_].split(','):
                    Parse_data[hash_.split("=")[0]] = hash_.split("=")[1]
                continue
            Parse_data[key_] = source_[key_] 
        data_.append(Parse_data)


In [108]:
event = data_[0]

In [ ]:
# initial execution

# Add a constraint
CREATE CONSTRAINT process_ProcessGuid IF NOT EXISTS FOR (p:Process) REQUIRE p.ProcessGuid IS UNIQUE;
CREATE CONSTRAINT user_LogonGuid IF NOT EXISTS FOR (p:User) REQUIRE p.LogonGuid IS UNIQUE;
CREATE CONSTRAINT computer_DeviceName IF NOT EXISTS FOR (c:Computer) REQUIRE c.DeviceName IS UNIQUE;
CREATE CONSTRAINT image_HashSHA256 IF NOT EXISTS FOR (i:Image) REQUIRE i.HashSHA256 IS UNIQUE;

In [111]:
event

{'ProviderName': 'Microsoft-Windows-Sysmon',
 'ProviderGUID': '',
 'EventID': '1',
 'Level': '4',
 'Task': '1',
 'Opcode': '0',
 'Keywords': '0x8000000000000000',
 'TimeCreated': '2025/09/10 12:31:40.133822200',
 'EventRecordID': '32441',
 'ActivityID': '',
 'RelatedActivityID': '',
 'ProcessID': '896',
 'ThreadID': '12148',
 'Channel': 'Microsoft-Windows-Sysmon/Operational',
 'Computer': 'DESKTOP-K05T3EJ',
 'UserID': 'S-1-5-18',
 'User': 'DESKTOP-K05T3EJ\\Sec User',
 'Version': '5',
 'Description': 'Host Process for Windows Tasks',
 'EventData': ['-',
  '2025-09-10 12:31:40.124',
  '{B2A55EDD-6FAC-68C1-DB05-000000000900}',
  '4972',
  'C:\\Windows\\System32\\taskhostw.exe',
  '10.0.26100.4768 (WinBuild.160101.0800)',
  'Host Process for Windows Tasks',
  'Microsoft® Windows® Operating System',
  'Microsoft Corporation',
  'taskhostw.exe',
  'taskhostw.exe',
  'C:\\WINDOWS\\system32\\',
  'DESKTOP-K05T3EJ\\Sec User',
  '{B2A55EDD-258E-68C0-1624-600100000000}',
  '0x0000000001602416',
 

In [141]:
# neo4j connectivity
from neo4j import GraphDatabase
URI = "neo4j://localhost"
AUTH = ("neo4j", "neo4j")

# with GraphDatabase.driver(URI, auth=AUTH) as driver:
#     driver.verify_connectivity()

In [161]:
# Create nodes for a single event
import random
import uuid

# print( random.ran)
event_id = "c48d2468_d70b_4454_b3e6_0b502c0a405a" #str(uuid.uuid4()).replace("-","_")

# template
# MERGE (computer:User {uid:'S-1-5-20'}) On CREATE SET user.title='NT AUTHORITY\\SYSTEM'
user_key=f"user_{event_id}"
computer_key=f"computer_{event_id}"

def get_value(v):
    return f"'{v}'"

def set_value_str(key_, event, title, other_fields):
    return f"{key_}.title='{event[title]}', "+", ".join([ f"{key_}.{set_field}='{event[set_field]}'" for set_field in other_fields])

print(f"MERGE ({computer_key}:Computer {{ Computer:{get_value(event['Computer'])}}}) ON CREATE SET {computer_key}.title={get_value(event['Computer'])}")
print(f"MERGE ({user_key}:User {{ LogonGuid:{get_value(event["LogonGuid"])}}}) ON CREATE SET {set_value_str(user_key, event, title="User" , other_fields=("UserID","LogonId"))}")



MERGE (computer_c48d2468_d70b_4454_b3e6_0b502c0a405a:Computer { Computer:'DESKTOP-K05T3EJ'}) ON CREATE SET computer_c48d2468_d70b_4454_b3e6_0b502c0a405a.title='DESKTOP-K05T3EJ'
MERGE (user_c48d2468_d70b_4454_b3e6_0b502c0a405a:User { LogonGuid:'b2a55edd-258e-68c0-1624-600100000000'}) ON CREATE SET user_c48d2468_d70b_4454_b3e6_0b502c0a405a.title='DESKTOP-K05T3EJ\Sec User', user_c48d2468_d70b_4454_b3e6_0b502c0a405a.UserID='S-1-5-18', user_c48d2468_d70b_4454_b3e6_0b502c0a405a.LogonId='0x1602416'


In [ ]:
# print(user_key)

#user_c48d2468_d70b_4454_b3e6_0b502c0a405a.UserID='S-1-5-18',      user_c48d2468_d70b_4454_b3e6_0b502c0a405a.LogonId='0x1602416'


print()

user_c48d2468_d70b_4454_b3e6_0b502c0a405a
user_c48d2468_d70b_4454_b3e6_0b502c0a405a.User='DESKTOP-K05T3EJ\Sec User',user_c48d2468_d70b_4454_b3e6_0b502c0a405a.UserID='S-1-5-18',user_c48d2468_d70b_4454_b3e6_0b502c0a405a.LogonId='0x1602416'


In [ ]:
# create nodes when a event is created.
# Create all the edges when a event is created.  

# nodes
    # Computer 
        # Computer (primary unique) : title
    # user
        # LogonGuid (primary unique)
        # UserID 
        # User :title
        # LogonId
    # process
        # ProcessGuid (primary unique), if it is a parent process, use ParentProcessGuid
        # ProcessId : title, if it is a parent pocess, use ParentProcessId
        # IntegrityLevel
        # CurrentDirectory
        # CommandLine
        # UtcTime
    # Image
        # Hash-sha256 (primary unique)
        # FilePath
        # Hash-md5
        # Hash-IMPHASH
        # OriginalFileName :title
        # FileVersion
        # Product
        # Company

# edges
    # created 
        # time
    # logged_in
        # LogonGuid 
        # LogonId
    # process_image

# semantics

user created process
user logged_in Computer 
process1 created process 2
process created_at computer
process  has_image image


# ContextComputer= source_['Computer']
# TimeCreated = source_['TimeCreated']
# UserID = source_['UserID']
# ProcessID = source_['ProcessID']


exe_properties = ["OriginalFileName","Product","Company","FileVersion","Image"]
user_properties = ["UserID"]
process_properties = ["ProcessGuid","CommandLine","CurrentDirectory","IntegrityLevel"]



In [53]:
len(data_[0])

4

In [48]:
# User created Process
print(f"User: {Parse_data['User']} -> created -> Process: {Parse_data["ProcessGuid"]}+{Parse_data['ProcessId']}")

User: NT AUTHORITY\SYSTEM -> created -> Process: {b2a55edd-3679-68c0-5e05-000000000900}+10524


In [ ]:
# retrive the data "process create" sysmon event
# parse and decide which data to add in the graph
# and construct a graph query to insert the data as nodes and edges 
    # use this file to create a relationships https://github.com/OTRF/OSSEM-DM/blob/main/use-cases/mitre_attack/attack_events_mapping.csv


In [ ]:




#To create a relevant node

# create process node
MERGE (Pid:Process {guid:"{b2a55edd-3679-68c0-5e05-000000000910}"}) ON CREATE SET Pid.OriginalFileName="taskhostw.exe"=2000, Pid.Product='Microsoft® Windows® Operating System',
Pid.title="taskhostw.exe",
Pid.Company='Microsoft Corporation',
Pid.Image='C:\\Windows\\System32\\taskhostw.exe'

#create user node, if we try to create another node with same uid: it will be ignored, and as we set the uid as unique
MERGE (user:User {uid:'S-1-5-20'}) On CREATE SET user.title='NT AUTHORITY\\SYSTEM'

#edge
MERGE (user)-[:CREATED]->(Pid)

#show the relationships in graph
MATCH (user:User {uid:'S-1-5-20'})
MATCH path = (user)-[:CREATED]->(p)
RETURN path;

# # or to view all the user's
# MATCH (user)
# MATCH path = (user)-[:CREATED]->(p)
# RETURN path;

Merge (ParetProcess)-[:ParentProcess] ->[pid]

MATCH (process:Process)
MATCH path= (user)-[:CREATED]->(process)
RETURN path;

SyntaxError: invalid syntax (3525626900.py, line 2)

In [ ]:
MERGE (user:User {uid:'S-1-5-19'})-[:CREATED]->(proces:Process {guid:"{b2a55edd-3679-68c0-5e05-000000000905}"})

#this actually creates a two new nodes and edges between them. 

MATCH (user:User {uid:'S-1-5-19'})
MATCH path = (user)-[:CREATED]->(p)
RETURN path;

#here (p) means, whatevery nodes with Created lable by this user.
